Import google drive for the data

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


First we convert the IOB file exported from the UBIAI annotation tool to spacy JSON:

In [ ]:
!python -m spacy convert /content/gdrive/MyDrive/dsci558/test.tsv ./ -t json -n 1 -c iob
!python -m spacy convert /content/gdrive/MyDrive/dsci558/train.tsv ./ -t json -n 1 -c iob

ℹ Auto-detected token-per-line NER format
⚠ Document delimiters found, automatic document segmentation with `-n`
disabled.
✔ Generated output file (1 documents): test.json
ℹ Auto-detected token-per-line NER format
⚠ Document delimiters found, automatic document segmentation with `-n`
disabled.
✔ Generated output file (1 documents): train.json


In [ ]:
# f = open('train.json','r')
# f.read()
# f.close()

After converting the training and dev files to JSON file, we need to convert them to spacy binary file:

In [ ]:
!python -m spacy convert /content/test.json ./ -t spacy
!python -m spacy convert /content/train.json ./ -t spacy

✔ Generated output file (11 documents): test.spacy
✔ Generated output file (77 documents): train.spacy


Next we install spacy and transformer library pipeline:

In [ ]:
!pip install -U spacy
!python -m spacy download en_core_web_trf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 460.3 MB 26 kB/s 
     |████████████████████████████████| 53 kB 2.1 MB/s 
     |████████████████████████████████| 4.7 MB 28.3 MB/s 
     |████████████████████████████████| 1.1 MB 51.9 MB/s 
     |████████████████████████████████| 6.6 MB 51.1 MB/s 
     |████████████████████████████████| 182 kB 62.7 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')


Next we install the cuda:

In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

--2022-11-18 22:17:29--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 152.199.20.126
Connecting to developer.nvidia.com (developer.nvidia.com)|152.199.20.126|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://developer.nvidia.com/downloads/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 [following]
--2022-11-18 22:17:29--  https://developer.nvidia.com/downloads/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?-DCCERZ5R8hvgcRfzuGwq1q4c1qEgkIYkpP1i5Y_WnBsVvCh0JVYdjHAf1AWn6eKml3q-RVRSpACQfZS5bnj6s1g-

Install pytorch. Install spacy and transformer packages:

In [ ]:
!pip install torch==1.7.1+cu92 torchvision==0.8.2+cu92 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [ ]:
!export CUDA_PATH="/usr/local/cuda-9.2"
!pip install -U spacy[cuda92,transformers]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Install cupy

In [ ]:
!pip install cupy-cuda92
# !pip install cupy-wheel
# !pip uninstall cupy-cuda11x

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached cupy_cuda92-9.6.0-cp37-cp37m-manylinux1_x86_64.whl (55.0 MB)


Set the cuda path

In [ ]:
!export LD_LIBRARY_PATH=$CUDA_PATH/lib64:$LD_LIBRARY_PATH

After updating the spacy config.cfg file with the training and test paths, we auto-fill the config file with the rest of the parameters that the BERT model will need

In [ ]:
!python -m spacy init fill-config /content/base_config.cfg config_spacy.cfg

✔ Auto-filled config with all values
✔ Saved config
config_spacy.cfg
You can now add your data and train your pipeline:
python -m spacy train config_spacy.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


Before launching the training, lets debug the config file to make sure everything is correct:

In [ ]:
!python -m spacy debug data /content/config_spacy.cfg


============================ Data file validation ============================
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
✔ Pipeline can be initialized with data
✔ Corpus is loadable

=============================== Training stats ===============================
Language: en
Training pipeline: transformer, ner
77 training docs
11 evaluat

Finally, we are ready to start the training:

In [ ]:
# train without gpu
!python -m spacy train /content/config_spacy.cfg --output ./  

ℹ Saving to output directory: .
ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

=========================== Initializing pipeline ===========================
[2022-11-15 05:27:09,258] [INFO] Set up nlp object from config
INFO:spacy:Set up nlp object from config
[2022-11-15 05:27:09,270] [INFO] Pipeline: ['transformer', 'ner']
INFO:spacy:Pipeline: ['transformer', 'ner']
[2022-11-15 05:27:09,273] [INFO] Created vocabulary
INFO:spacy:Created vocabulary
[2022-11-15 05:27:09,274] [INFO] Finished initializing nlp object
INFO:spacy:Finished initializing nlp object
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceCl

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install -U spacy
!python -m spacy download en_core_web_trf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 460.3 MB 26 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')


In [ ]:
import spacy

In [ ]:
!pip install spacy-transformers

In [ ]:
import spacy_transformers

In [ ]:
nlp = spacy.load("/content/gdrive/MyDrive/dsci558/model_best/content/model-best")

# text = [
# '''Qualifications
# - A thorough understanding of C# and .NET Core
# - Knowledge of good database design and usage
# - An understanding of NoSQL principles
# - Excellent problem solving and critical thinking skills
# - Curious about new technologies
# - Experience building cloud hosted, scalable web services
# - Azure experience is a plus
# Requirements
# - Bachelor's degree in Computer Science or related field
# (Equivalent experience can substitute for earned educational qualifications)
# - Minimum 4 years experience with C# and .NET
# - Minimum 4 years overall experience in developing commercial software
# '''
# ]

text = [
    '''pursuing a Bachelor’s in Statistics, Math or Computer Science,
    Coding proficiency in SQL.'''
]
for doc in nlp.pipe(text, disable=["tagger", "parser"]):
    print(f"spans: {[(e.start, e.text, e.label_) for e in doc.ents]}")

spans: [(2, 'Bachelor', 'DIPLOMA'), (5, 'Statistics', 'DIPLOMA_MAJOR'), (7, 'Math', 'DIPLOMA_MAJOR'), (9, 'Computer Science', 'DIPLOMA_MAJOR'), (16, 'SQL', 'SKILLS'), (17, '.', 'SKILLS')]


In [ ]:
import pandas as pd

job_df = pd.read_csv('job_data_no_dup.csv')

In [ ]:
job_df.head()

,source,job_title,company_name,location,job_details,job_requirement,salary,work_type,url,job_category,company_url
0,glassdoor,Data Scientist,Walmart,"Sunnyvale, CA",Position Summary...\nWhat you'll do...\nData S...,": 7 years' experience in data science, machine...",$106K - $155K (Glassdoor est.),NaN,https://www.glassdoor.com/partner/jobListing.h...,Data Scientist,https://www.glassdoor.com/Overview/Working-at-...
1,glassdoor,Mid-Level Data Scientist - Financial Services ...,USAA,"San Antonio, TX",Purpose of Job USAA knows what it means to ser...,\n\nidentifies and manages existing and emergi...,NaN,NaN,https://www.glassdoor.com/partner/jobListing.h...,Data Scientist,https://www.glassdoor.com/Overview/Working-at-...
2,glassdoor,Data Scientist,Driven Brands,"Benicia, CA",We invite you to join us at Driven Brands!\nHe...,"\nidentify, analyze & interpret trends in comp...",$83K - $122K (Glassdoor est.),NaN,https://www.glassdoor.com/partner/jobListing.h...,Data Scientist,https://www.glassdoor.com/Overview/Working-at-...
3,glassdoor,Data Scientist,CAPITAL Services,"Sioux Falls, SD",Job Title: Data Scientist\n*\nSummary*: CAPITA...,.\nexperienced manipulating large data sources...,$77K - $127K (Glassdoor est.),full-time,https://www.glassdoor.com/partner/jobListing.h...,Data Scientist,https://www.glassdoor.com/Overview/Working-at-...
4,glassdoor,Remote Senior Data Scientist NLP,First American Financial Corporation,"Santa Ana, CA",Company Summary\nJoin a team that puts its Peo...,/technology used\nexperience with state-of-the...,$90K - $138K (Glassdoor est.),NaN,https://www.glassdoor.com/partner/jobListing.h...,Data Scientist,https://www.glassdoor.com/Overview/Working-at-...


In [ ]:
job_df = job_df.fillna('')

In [ ]:
def info_extraction(req):
  for doc in nlp.pipe(req, disable=["tagger", "parser"]):
    return ([(ent.text, ent.label_) for ent in doc.ents])

In [ ]:
job_req_list = job_df['job_requirement'].tolist()

In [ ]:
req_dic_list = []
label_set = set()
for req in job_req_list:
  req_dic = {}
  res = info_extraction([req])
  # print(res)
  for (v,k) in res:
    label_set.add(k)
    if k not in req_dic:
      req_dic[k] = [v]
    if k in req_dic:
      req_dic[k].append(v)
  req_dic_list.append(req_dic)

In [ ]:
req_dic_list[:5]

In [ ]:
import re
import string
puncs = string.punctuation
def compile_requirement(label,outputlist):
  for req in req_dic_list:
    if label in req:
      attributes = set(req[label]) # remove the duplications
      if "'s" in attributes:
        attributes.remove("'s")
      for p in puncs:
        if p in attributes:
          attributes.remove(p) # remove the punctuations
      attributes = ';'.join(list(attributes))
    else:
      attributes = ''
    outputlist.append(attributes)
  return outputlist

In [ ]:
skill_list = []
diploma_list = []
diploma_major_list = []
exp_list = []

skill_list = compile_requirement('SKILLS',[])
diploma_list = compile_requirement('DIPLOMA',[])
diploma_major_list = compile_requirement('DIPLOMA_MAJOR',[])
exp_list = compile_requirement('EXPERIENCE',[])
    

In [ ]:
diploma_list

In [ ]:
job_df['DIPLOMA'] = diploma_list
job_df['SKILLS'] = skill_list
job_df['DIPLOMA_MAJOR'] = diploma_major_list
job_df['EXPERIENCE'] = exp_list

In [ ]:
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

In [ ]:
s = 'puerto rico'
s.title()

'Puerto Rico'

In [ ]:
def transform_location(x):
  location = x.split(',')
  if len(location)>1:
    state = location[-1].strip(' ')
    # print(state)
    city = ','.join(location[:-1])
    # if state in us_state_to_abbrev:
    #   print(state)
    #   state = us_state_to_abbrev[state]
  else:
    state = location[0]
    city = ''
  if state.title() in us_state_to_abbrev:
      # print(state)
      state = us_state_to_abbrev[state.title()]
      return state, city
  for s in us_state_to_abbrev:
    if s.lower() in state.lower():
      state = us_state_to_abbrev[s]
      return state, city
    if us_state_to_abbrev[s] == state.upper():
      state = us_state_to_abbrev[s]
      return state, city
  return state,city

In [ ]:
import pandas as pd
# glassdoor_company_df = pd.read_csv('/content/company_data_1122.csv')
linkedin_company_df = pd.read_csv('/content/linkedin_company.csv')

In [ ]:
# glassdoor_company_df[glassdoor_company_df['state/area']=='061']['']
# glassdoor_company_df = glassdoor_company_df.replace('Monmouth Junction, 061','Monmouth Junction, NJ')

In [ ]:
# glassdoor_company_df = glassdoor_company_df.fillna('')
linkedin_company_df = linkedin_company_df.fillna('')

In [ ]:
linkedin_company_df.tail()

,company_name,headquarters,website,size,industry,founded,description,url,img_url
2675,DaVita,,,,,,,,
2676,Goodyear Commercial Tire & Service Centers (CTSC),,,,,,,,
2677,Skin Research Institute,,,,,,,,
2678,"Finches, LLC",,,,,,,,
2679,Act! LLC,,,,,,,,


In [ ]:
states = []
cities = []
for i in range(len(linkedin_company_df['headquarters'])):
  x = linkedin_company_df['headquarters'][i]
  # print(transform_location(x))
  state, city = transform_location(x)
  states.append(state)
  cities.append(city)

In [ ]:
# glassdoor_company_df['state/area'] = states
# glassdoor_company_df['city'] = cities
linkedin_company_df['state/area'] = states
linkedin_company_df['city'] = cities

In [ ]:
linkedin_company_df.head()

,company_name,headquarters,website,size,industry,founded,description,url,img_url,state/area,city
0,KBI Biopharma,"Durham, NC",http://www.kbibiopharma.com,"1,001-5,000 employees",Biotechnology Research,1996.0,KBI Biopharma is an award-winning biopharmaceu...,https://www.linkedin.com/company/kbi-biopharma...,https://media-exp1.licdn.com/dms/image/C4D0BAQ...,NC,Durham
1,DT Professional Services,"Los Angeles, California",http://www.dtprosusa.com,11-50 employees,IT Services and IT Consulting,2013.0,DT Professional Services is a SBA 8(a) and HUB...,https://www.linkedin.com/company/dt-profession...,https://media-exp1.licdn.com/dms/image/C560BAQ...,CA,Los Angeles
2,Activision Blizzard Media,"San Francisco, CA",http://www.activisionblizzardmedia.com/,51-200 employees,Advertising Services,,Activision Blizzard Media is the gateway for b...,https://www.linkedin.com/company/activision-bl...,https://media-exp1.licdn.com/dms/image/C4E0BAQ...,CA,San Francisco
3,"Voloridge Investment Management, LLC","Jupiter, FL",http://www.voloridge.com,51-200 employees,Financial Services,2009.0,Voloridge Investment Management was launched i...,https://www.linkedin.com/company/voloridge-inv...,https://media-exp1.licdn.com/dms/image/C4E0BAQ...,FL,Jupiter
4,Prudential Financial,"Newark, New Jersey",http://www.prudential.com,"10,001+ employees",Financial Services,,Prudential Financial (NYSE:PRU) was founded on...,https://www.linkedin.com/company/prudential-fi...,https://media-exp1.licdn.com/dms/image/C4E0BAQ...,NJ,Newark


In [ ]:
# glassdoor_company_df = glassdoor_company_df.drop(columns=['state/area','city'])

# linkedin_company_df = linkedin_company_df.drop(columns=['state/area','city'])
linkedin_company_df[linkedin_company_df['state/area']=='AB']
# linkedin_company_df = linkedin_company_df.replace('青岛市, 山东省','qingdao, shandong')

# linkedin_company_df = linkedin_company_df.replace('CH-1009 PULLY/LAUSANNE, -','CH-1009 PULLY/LAUSANNE')

# linkedin_company_df = linkedin_company_df.replace('MA, 01824','MA')
# linkedin_company_df = linkedin_company_df.replace('Minnesota, 55077','Minnesota')
# linkedin_company_df = linkedin_company_df.replace('Holland, MI, 49423','Holland, MI')
# linkedin_company_df = linkedin_company_df.replace('Jacksonville, FL., .','Jacksonville, FL')

,company_name,headquarters,website,size,industry,founded,description,url,img_url,state/area,city
1131,International Cooling Tower Inc. (ICT),"Edmonton, AB",http://www.ictower.com,"501-1,000 employees",Construction,1958.0,"About Us \n\nInternational Cooling Tower, Inc....",https://ca.linkedin.com/company/international-...,https://media-exp1.licdn.com/dms/image/C4E0BAQ...,AB,Edmonton


In [ ]:
# print(job_df['location'].value_counts())
# dict(glassdoor_company_df['state/area'].value_counts()).keys()

dict(linkedin_company_df['state/area'].value_counts()).keys()

dict_keys(['CA', '', 'TX', 'NY', 'IL', 'VA', 'FL', 'MA', 'PA', 'OH', 'NJ', 'GA', 'NC', 'MI', 'MD', 'WA', 'MN', 'CO', 'MO', 'TN', 'WI', 'AZ', 'DC', 'CT', 'IN', 'UT', 'OR', 'England', 'IA', 'RI', 'OK', 'SC', 'AR', 'Ontario', 'NH', 'LA', 'KY', 'KS', 'ID', 'London', 'AL', 'NV', 'Quebec', 'Maharashtra', 'Tokyo', 'NE', 'SD', 'NM', 'Zurich', 'D.C.', 'British Columbia', 'WY', 'North Rhine-Westphalia', 'NSW', 'Île-de-France', 'DE', 'MT', 'ME', 'Switzerland', 'Paris', 'MS', 'HI', 'Rhineland-Palatinate', 'Ireland', 'Alberta', 'AK', 'Cambridgeshire', 'Gujarat', 'ND', 'Baden-Württemberg', 'Karnataka', 'Québec', 'Skane County', 'United Kingdom', 'Singapore', 'Delhi', 'Worldwide', 'France', 'Northern Ireland', 'Western Australia', 'Auvergne-Rhône-Alpes', 'NewJersey', 'Catalonia', 'Israel', 'Geneva', 'Gauteng', 'Lesser Poland', 'FI', 'North Holland', 'Middlesex', 'Zuid-Holland', 'UK', 'County Cork', 'Victoria', 'Oxfordshire', 'Uttar Pradesh', 'BC', 'Tamilnadu', 'Caifornia', 'Nordrhein-Westfalen', 'Buc

In [ ]:
# glassdoor_company_df.head()
# glassdoor_company_df.to_csv('glassdoor_company.csv',encoding='utf-8-sig', index=False)
linkedin_company_df.to_csv('linkedin_company_update.csv',encoding='utf-8-sig', index=False)

In [ ]:
linkedin_company_df

,company_name,headquarters,website,size,industry,description,url,state/area,city
0,KBI Biopharma,"Durham, NC",http://www.kbibiopharma.com,"1,001-5,000 employees",Biotechnology Research,KBI Biopharma is an award-winning biopharmaceu...,https://www.linkedin.com/company/kbi-biopharma...,NC,Durham
1,DT Professional Services,"Los Angeles, California",http://www.dtprosusa.com,11-50 employees,IT Services and IT Consulting,DT Professional Services is a SBA 8(a) and HUB...,https://www.linkedin.com/company/dt-profession...,CA,Los Angeles
2,Activision Blizzard Media,"San Francisco, CA",http://www.activisionblizzardmedia.com/,51-200 employees,Advertising Services,Activision Blizzard Media is the gateway for b...,https://www.linkedin.com/company/activision-bl...,CA,San Francisco
3,"Voloridge Investment Management, LLC","Jupiter, FL",http://www.voloridge.com,51-200 employees,Financial Services,Voloridge Investment Management was launched i...,https://www.linkedin.com/company/voloridge-inv...,FL,Jupiter
4,Santee Cooper,"Moncks Corner, SC",http://www.santeecooper.com,"1,001-5,000 employees",Utilities,Santee Cooper is South Carolina’s largest powe...,https://www.linkedin.com/company/santeecooper/...,SC,Moncks Corner
...,...,...,...,...,...,...,...,...,...
3008,Newsela,,,,,,,,
3009,Act! LLC,,,,,,,,
3010,Nautica,,,,,,,,
3011,C.H. Robinson,,,,,,,,


In [ ]:
# glassdoor_company_df[glassdoor_company_df['state/area']=='061']

In [ ]:
job_df.to_csv('job_req.csv',index=False,encoding='utf-8-sig')

In [ ]:
glassdoor_df = job_df[job_df['source']=='glassdoor']
# glassdoor_df

In [ ]:
glassdoor_df.to_csv('glassdoor_req.csv',index=False)

In [ ]:
linkedin_df = job_df[job_df['source']=='linkedin']

In [ ]:
linkedin_df.to_csv('linkedin_req.csv',index=False)

In [ ]:
from google.colab import files
files.download('linkedin_req.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
job_df.to_csv('job_req.csv',index=False)